In [6]:
import pandas as pd

In [7]:
accidents_df = pd.read_csv("D:/Programing/Weather2Go2/US_Accidents_March23.csv/US_Accidents_March23.csv")
accidents_df.shape


(7728394, 46)

In [8]:
weather_df = pd.read_csv("D:/Programing/Weather2Go2/Data/mi_all_hourly_2016_2023.csv")
weather_df.shape

(1757232, 10)

In [9]:
CATS = [
  "clear","cloudy","fog","rain_light","rain_heavy","snow_light","snow_heavy",
  "freezing_rain","sleet_mix","thunder","hail","smoke_dust","severe","other","unknown"
]

def kaggle_weather_to_cat(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return "unknown"

    s = str(x).strip().lower()

    # remove common noise tokens but keep meaning
    # (we ignore "windy" as its own category; wind is handled by wind_speed feature)
    s = s.replace("/ windy", "").replace("windy", "").strip()

    # --- highest priority: truly severe phenomena ---
    if "tornado" in s or "funnel cloud" in s:
        return "severe"
    if "squall" in s:
        return "severe"
    if "duststorm" in s:
        return "severe"

    # --- hail (including small/light/heavy) ---
    if "hail" in s:
        return "hail"

    # --- thunder / t-storm ---
    if ("thunder" in s) or ("t-storm" in s) or ("tstorm" in s) or ("thunderstorm" in s):
        return "thunder"

    # --- freezing precip (most dangerous) ---
    # freezing drizzle/rain/fog -> treat as freezing risk bucket
    if "freezing" in s:
        return "freezing_rain"

    # --- sleet / ice pellets / wintry mix ---
    if ("sleet" in s) or ("ice pellet" in s) or ("wintry mix" in s) or ("rain and sleet" in s):
        return "sleet_mix"

    # --- snow (heavy vs light) ---
    if "snow" in s or "blowing snow" in s or "drifting snow" in s or "snow grains" in s:
        if "heavy" in s:
            return "snow_heavy"
        return "snow_light"

    # --- rain/drizzle/showers (heavy vs light) ---
    if ("rain" in s) or ("drizzle" in s) or ("shower" in s):
        if "heavy" in s:
            return "rain_heavy"
        return "rain_light"

    # --- fog/mist/haze ---
    if ("fog" in s) or ("mist" in s) or ("haze" in s):
        return "fog"

    # --- smoke/dust/sand/ash ---
    if ("smoke" in s) or ("dust" in s) or ("sand" in s) or ("ash" in s) or ("volcanic" in s):
        return "smoke_dust"

    # --- clouds ---
    if ("overcast" in s) or ("cloudy" in s) or ("scattered clouds" in s) or ("partly cloudy" in s) or ("mostly cloudy" in s):
        return "cloudy"

    # --- clear/fair ---
    if ("clear" in s) or (s == "fair") or s.startswith("fair "):
        return "clear"

    # --- explicit "n/a precipitation" should not force rain ---
    if "n/a precipitation" in s:
        return "other"

    return "other"


In [10]:
def wmo_to_category(code: int) -> str:
    if code in [0]:
        return "clear"

    if code in [1, 2, 3]:
        return "cloudy"

    if code in [45, 48]:
        return "fog"

    # drizzle
    if code in [51, 53, 55]:
        return "rain_light"
    if code in [56, 57]:
        return "freezing_rain"

    # rain
    if code in [61, 63]:
        return "rain_light"
    if code in [65]:
        return "rain_heavy"
    if code in [66, 67]:
        return "freezing_rain"

    # snow
    if code in [71, 73, 77]:
        return "snow_light"
    if code in [75]:
        return "snow_heavy"

    # showers
    if code in [80, 81]:
        return "rain_light"
    if code in [82]:
        return "rain_heavy"
    if code in [85]:
        return "snow_light"
    if code in [86]:
        return "snow_heavy"

    # thunderstorms
    if code in [95]:
        return "thunder"
    if code in [96, 99]:
        return "hail"

    return "other"
